##### my-zero-to-gbm-proj-assign

In [3]:
import pandas as pd
import numpy as np

In [4]:
inputs = pd.read_csv('inputs.csv',index_col=0)
targets = pd.read_csv('targets.csv',index_col=0)

In [5]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [6]:
from sklearn.metrics import r2_score
# def rmspe(y_true, y_pred):
#     return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100

In [7]:
from sklearn.model_selection import ShuffleSplit, RepeatedKFold

In [8]:
# ss = ShuffleSplit(n_splits = 5, test_size = 0.25, random_state=11111)
ss = RepeatedKFold(n_splits=2, n_repeats=2, random_state=11111)

In [9]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(3)
poly_inputs = pd.DataFrame(poly.fit_transform(inputs)).astype('float32')

In [10]:
from sklearn.metrics import mean_squared_error
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100

In [29]:
from sklearn.ensemble import HistGradientBoostingRegressor

In [12]:
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [13]:
# explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import HalvingGridSearchCV

In [30]:
def train_and_evaluate(X_train, train_targets, X_val, val_targets, **params):
#    model = LGBMRegressor(random_state=11111,silent=False,first_metric_only= True,metric = "None",feval=[rmspe],force_col_wise=True,mc=[0,1,1,1,1,1,1,1,1,1],**params)
    model = HistGradientBoostingRegressor(random_state=11111,monotonic_cst=[0,1,1,1,1,1,1,1,1,1],verbose=1)
    model.fit(X_train, train_targets)#,early_stopping_rounds=5,eval_set=(X_val,val_targets))
    pred_train = model.predict(X_train)
    pred_val = model.predict(X_val)
    train_R2 = round(r2_score(train_targets,pred_train ),3)
    train_RMSPE = round(mean_squared_error(train_targets, pred_train,squared=False),3)
    val_R2 = round(r2_score(val_targets, pred_val),3)
    val_RMSPE = round(mean_squared_error(val_targets, pred_val,squared=False),3)

    # train_rmse = rmse(model.predict(X_train), train_targets)
    # val_rmse = rmse(model.predict(X_val), val_targets)
    return model, train_R2, train_RMSPE, val_R2, val_RMSPE
#n_jobs = -1 means that use all the available threads in that machine where the alogorithm is running 

In [35]:
modelc.get_params().keys()

dict_keys(['categorical_features', 'early_stopping', 'l2_regularization', 'learning_rate', 'loss', 'max_bins', 'max_depth', 'max_iter', 'max_leaf_nodes', 'min_samples_leaf', 'monotonic_cst', 'n_iter_no_change', 'random_state', 'scoring', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [38]:
# #modelc = LGBMRegressor(random_state=11111,silent=False,first_metric_only= True,metric = "None",force_col_wise=True,mc=[0,1,1,1,1,1,1,1,1,1])#,bagging_freq= 5, bagging_fraction= 0.75)
modelc = HistGradientBoostingRegressor(random_state=11111,monotonic_cst=[0,1,1,1,1,1,1,1,1,1],verbose=1)
param_grid = {"loss": ['squared_error','absolute_error'],
              "learning_rate" : [0.1,0.01,0.001],
              "max_iter": [100,300],
              "max_leaf_nodes": [16,64,256],
              "min_samples_leaf": [10,20,30],
              "l2_regularization": [0.0,0.001,0.01],
              "max_bins": [63,255],
              "n_iter_no_change": [10,20,50],
              "validation_fraction": [0.1,0.5,1.0],
              "warm_start": [True,False]
            }

In [21]:
# param_grid = {"boosting_type": ['dart'],
#               "learning_rate" : [0.01,0.05,0.1],
#               "n_estimators": [1000],
#               "min_data_in_leaf": [0,20],
#               "min_split_gain": [0.001],
#               "objective": ["huber"],
#               "num_leaves": [16,64,256,1024],
#               "max_bin": [63,255,1024],
#               "min_sum_hessian_in_leaf": [1.0,100.0],
#               "reg_lambda": [0.01,10.0],
#               "uniform_drop": [True],
#               "monotone_constraints_method": ['advanced'] ,
#               "linear_tree": ['true'],
#               "data_random_seed": [42],
#             }

In [27]:
# models = []
# model_params = []
# grid_params = []
# for train_idxs, val_idxs in ss.split(poly_inputs):
#    X_train, train_targets = poly_inputs.iloc[train_idxs], targets.iloc[train_idxs]
#    X_val, val_targets = poly_inputs.iloc[val_idxs], targets.iloc[val_idxs]
#    X_train = np.ascontiguousarray(X_train).reshape(-1,10)
#    train_targets = np.ascontiguousarray(train_targets).ravel()
#    X_val = np.ascontiguousarray(X_val).reshape(-1,10)
#    val_targets = np.ascontiguousarray(val_targets).ravel()
#    grid_search = HalvingGridSearchCV(modelc, param_grid,verbose=1,return_train_score=True,random_state=11111,factor=2,min_resources=1000,error_score='raise').fit(X_train, train_targets)
# #   grid_search = GridSearchCV(modelc, param_grid,verbose=1,return_train_score=True,error_score='raise').fit(X_train, train_targets)
#    model, train_R2, train_RMSPE, val_R2, val_RMSPE = train_and_evaluate(X_train, 
#                                                    train_targets, 
#                                                    X_val, 
#                                                    val_targets, 
#                                                    **grid_search.best_params_)
#    models.append(model)
#    model_params.append(model.get_params())
#    grid_params.append(grid_search.best_params_)
#    print('Train R2: {}, Train RMSPE: {}, Validation R2: {}, Validation RMSPE: {}'.format(train_R2, train_RMSPE, val_R2, val_RMSPE))

In [39]:
models = []
model_params = []
grid_params = []
for train_idxs, val_idxs in ss.split(poly_inputs):
   X_train, train_targets = poly_inputs.iloc[train_idxs], targets.iloc[train_idxs]
   X_val, val_targets = poly_inputs.iloc[val_idxs], targets.iloc[val_idxs]
   X_train = np.ascontiguousarray(X_train).reshape(-1,10)
   train_targets = np.ascontiguousarray(train_targets).ravel()
   X_val = np.ascontiguousarray(X_val).reshape(-1,10)
   val_targets = np.ascontiguousarray(val_targets).ravel()
   grid_search =    HalvingRandomSearchCV(modelc, param_grid,verbose=1,return_train_score=True,random_state=11111,factor=2,min_resources=1000,error_score='raise').fit(X_train, train_targets)
#   grid_search = GridSearchCV(modelc, param_grid,verbose=1,return_train_score=True,error_score='raise').fit(X_train, train_targets)
   model, train_R2, train_RMSPE, val_R2, val_RMSPE = train_and_evaluate(X_train, 
                                                   train_targets, 
                                                   X_val, 
                                                   val_targets, 
                                                   **grid_search.best_params_)
   models.append(model)
   model_params.append(model.get_params())
   grid_params.append(grid_search.best_params_)
   print('Train R2: {}, Train RMSPE: {}, Validation R2: {}, Validation RMSPE: {}'.format(train_R2, train_RMSPE, val_R2, val_RMSPE))

n_iterations: 8
n_required_iterations: 8
n_possible_iterations: 8
min_resources_: 1000
max_resources_: 214466
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 214
n_resources: 1000
Fitting 5 folds for each of 214 candidates, totalling 1070 fits
Binning 0.000 GB of training data: 0.006 s
Fitting gradient boosted rounds:
[1/300] 1 tree, 24 leaves, max depth = 6, in 0.013s
[2/300] 1 tree, 24 leaves, max depth = 6, in 0.014s
[3/300] 1 tree, 25 leaves, max depth = 6, in 0.014s
[4/300] 1 tree, 26 leaves, max depth = 7, in 0.016s
[5/300] 1 tree, 25 leaves, max depth = 6, in 0.016s
[6/300] 1 tree, 24 leaves, max depth = 6, in 0.015s
[7/300] 1 tree, 24 leaves, max depth = 6, in 0.016s
[8/300] 1 tree, 24 leaves, max depth = 6, in 0.015s
[9/300] 1 tree, 25 leaves, max depth = 7, in 0.018s
[10/300] 1 tree, 25 leaves, max depth = 7, in 0.016s
[11/300] 1 tree, 24 leaves, max depth = 7, in 0.015s
[12/300] 1 tree, 24 leaves, max depth = 7, in 0.015s
[13/300] 1 tree, 26 leaves, 

In [ ]:
model.feature_importances_

In [ ]:
df_gd_par = pd.DataFrame(grid_params)
df_md_par = pd.DataFrame(model_params)

In [ ]:
df_gd_par

In [ ]:
df_md_par